In [1]:
# %pip install statsmodels
# %pip install mlxtend
# %pip install xgboost

In [2]:
# Import necessary libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from sklearn.model_selection import train_test_split
import itertools
import random
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import xgboost as xgb
from sklearn.model_selection import train_test_split
import random
import pandas as pd

C:\Users\austinsh\AppData\Local\Temp\ipykernel_18976\526159370.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:

df_TagDesc = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\II Data\1 Collection\CSV\Not for Processing\TagDesc.csv')

# df_All_1 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_1o2.csv')
# df_All_2 = pd.read_csv(r'C:\Users\saust\OneDrive\Desktop\GitRepo\Project-OptiC4\1 Preprocess\Merge Data\contData_all_Avg_2o2.csv')
# # Concatenate (union) the dataframes
# df_All = pd.concat([df_All_1, df_All_2], ignore_index=True)

df_All = pd.read_csv(r'C:\Users\austinsh\Project-OptiC4\III Models\3 Model C4\3 Feature Selection\filtered-out_5-9_corr.csv')

print(df_All.head())

                  Date  425_pct_Al     Al2O3  M_Value  C4_pct_Eth  C4_pct_H2O  \
0  2012-05-17 09:00:00    6.319560  11.41670  3.50773    2.554580     22.0531   
1  2012-05-17 10:00:00    6.319915  11.40835  3.50797    2.555935     22.0557   
2  2012-05-17 12:00:00    6.320970  11.39165  3.50869    2.560005     22.0636   
3  2012-05-18 08:00:00    6.328690  11.39165  3.51401    2.540370     22.0604   
4  2012-05-18 09:00:00    6.336060  11.40500  3.51910    2.514430     22.0484   

   HydWtr_pct_Ammonia  C4_pct_Hex  HydWtr_Na2O  Butanol  ...   LC55568  \
0            0.867508    0.670721     3.354160  30.3662  ...  40.25250   
1            0.864729    0.671059     3.306475  30.3662  ...  40.25250   
2            0.856391    0.672071     3.163420  30.6785  ...  40.25250   
3            0.811652    0.674174     2.238130  32.6454  ...  39.88364   
4            0.773793    0.675408     1.391475  32.6454  ...  39.88364   

     LC90366   LC90368   PI55004   PI55020   TC55552   TC55553   TC5

In [4]:
# Set max columns to display
pd.set_option('display.max_columns', None)

In [5]:
# df_All = df_All[df_All['Date'] > '2022-06-15 00:00:00']

In [6]:



# # List of columns to exclude to run XGboost feature selection
# exclude_columns = ['Octanol', 'Hexanol',
#        'Ethanol', 'Decanol',
       
#        'TI52014', 'TI55013', 'TI55014', 'TI55015', 'TI55016', 'TI55017', 'TI55023',
#        # , 'TI55021'

#        'TC52015', 'FC52018', 'II52554', 'TI40050', 'VI52558B'

#        # 'FC55102', 'FC55152', 'LC55557', 'LC55568', 'TC55555',

#        # '425 SAO Al', 'FFC55553', 'LC52572', 'LC90366',

#        # 'FC42428', 'LC55553',

#        # 'FC55009'
#                    ]

# # Create a new DataFrame without the excluded columnsd
# df_All = df_All.drop(columns=exclude_columns)

In [7]:
df_All.columns

Index(['Date', '425_pct_Al', 'Al2O3', 'M_Value', 'C4_pct_Eth', 'C4_pct_H2O',
       'HydWtr_pct_Ammonia', 'C4_pct_Hex', 'HydWtr_Na2O', 'Butanol', 'DI55102',
       'DI55152', 'DI55580', 'FC55003', 'FC55552', 'FC55569', 'FC55576',
       'FFC55553', 'FFC55555', 'LC52572', 'LC55553', 'LC55557', 'LC55568',
       'LC90366', 'LC90368', 'PI55004', 'PI55020', 'TC55552', 'TC55553',
       'TC55555', 'TI40050', 'TI55021'],
      dtype='object')

In [8]:
# # Splitting into train and test
# X = df_All.drop('Butanol', axis=1)  # Assuming 'target' is your target column
# y = df_All['Butanol']
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
def iterate_feature_rotations(df_all, target_column, test_size=0.2, random_state=42, num_random_iterations=30):
    results = []
    columns = [col for col in df_all.columns if col != target_column and col != 'Date']
    random.seed(random_state)  # for reproducibility

    for feature in columns:
        for _ in range(num_random_iterations):
            # Randomly order the remaining features
            remaining_features = [f for f in columns if f != feature]
            random.shuffle(remaining_features)

            # Create a new ordered list of features
            ordered_features = [feature] + remaining_features

            reordered_df = df_all[ordered_features + [target_column]]

            # Splitting into train and test for each permutation
            X = reordered_df.drop(target_column, axis=1)
            y = reordered_df[target_column]
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

            # Create and fit the XGBoost model
            model = xgb.XGBRegressor(objective='reg:squarederror')
            model.fit(X_train, y_train)

            # Extract feature importances
            feature_importances = model.get_booster().get_score(importance_type="weight")

            # Store the result with the permutation order and feature importances
            results.append((ordered_features, feature_importances))

    return results


results = iterate_feature_rotations(df_All, 'Butanol')

# Convert results to a DataFrame
flattened_results = []
for ordered_features, importances in results:
    for feature, importance in importances.items():
        flattened_results.append({
            'Feature Rotation': ordered_features,
            'Feature': feature,
            'Importance': importance
        })

results_df = pd.DataFrame(flattened_results)


In [10]:
print(results_df)

                                        Feature Rotation     Feature  \
0      [425_pct_Al, LC55557, FFC55553, FC55003, PI550...  425_pct_Al   
1      [425_pct_Al, LC55557, FFC55553, FC55003, PI550...     LC55557   
2      [425_pct_Al, LC55557, FFC55553, FC55003, PI550...    FFC55553   
3      [425_pct_Al, LC55557, FFC55553, FC55003, PI550...     FC55003   
4      [425_pct_Al, LC55557, FFC55553, FC55003, PI550...     PI55004   
...                                                  ...         ...   
26995  [TI55021, M_Value, DI55102, 425_pct_Al, FC5557...     PI55004   
26996  [TI55021, M_Value, DI55102, 425_pct_Al, FC5557...     DI55580   
26997  [TI55021, M_Value, DI55102, 425_pct_Al, FC5557...  C4_pct_Hex   
26998  [TI55021, M_Value, DI55102, 425_pct_Al, FC5557...     LC90366   
26999  [TI55021, M_Value, DI55102, 425_pct_Al, FC5557...     TC55553   

       Importance  
0           497.0  
1           284.0  
2           217.0  
3           226.0  
4           174.0  
...           .

In [11]:
# Group by 'Feature' and calculate the average importance
average_importances = results_df.groupby('Feature')['Importance'].mean()

# Convert the Series to a DataFrame
average_importances_df = average_importances.reset_index()

# Rename the columns for clarity
average_importances_df.columns = ['Feature', 'Average Importance']

# Sort the DataFrame by 'Average Importance' in descending order
average_importances_df = average_importances_df.sort_values(by='Average Importance', ascending=False)

# # Display or save the DataFrame
# print(average_importances_df)
# # Or save it to a CSV file
# # average_importances_df.to_csv('average_feature_importances.csv', index=False)


In [12]:
# Merge the average_importances_df with df_TagDesc
# Assuming 'ID' in df_TagDesc corresponds to 'Feature' in average_importances_df
merged_df = average_importances_df.merge(df_TagDesc, left_on='Feature', right_on='ID', how='left')

# Select only the required columns
final_df = merged_df[['Feature', 'Average Importance', 'DESCRIPTION']]

# Rename the 'DESCRIPTION' column to 'Description'
final_df.rename(columns={'DESCRIPTION': 'Description'}, inplace=True)

# Display or save the DataFrame
print(final_df)
# Or save it to a CSV file
# final_df.to_csv('average_feature_importances_with_descriptions.csv', index=False)


               Feature  Average Importance                Description
0          HydWtr_Na2O          223.136667                        NaN
1   HydWtr_pct_Ammonia          215.513333                        NaN
2           425_pct_Al          208.247778                        NaN
3              FC55569          199.868889     30# STM TO C4 STRIPPER
4           C4_pct_H2O          193.913333                        NaN
5              LC55557          193.154444        FA-554 SLURRY LEVEL
6              TC55552          192.710000   DC-551 ALKOX  FD PREHEAT
7           C4_pct_Hex          189.655556                        NaN
8              DI55102          186.996667           HYDROL RX OUTLET
9              FC55552          184.591111   ALK FD TO HYDR RX DC-551
10          C4_pct_Eth          184.134444                        NaN
11             FC55003          182.184444   DA-551 O/H H2O TO DC-551
12             FC55576          180.901111   CONDENSATE TO DESUPERHTR
13             LC903

C:\Users\austinsh\AppData\Local\Temp\ipykernel_18976\3091506829.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df.rename(columns={'DESCRIPTION': 'Description'}, inplace=True)


In [13]:
from datetime import datetime

# Get the current date and time
current_date_time = datetime.now()

# Print the current date and time
print(current_date_time)


2024-05-06 21:14:57.027830
